In [ ]:
import os
import csv
import time
import yaml
from pathlib import Path
import _pickle as cpickle

import scanpy as sc
import torch
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import f1_score

from scSLAT.model import load_anndatas, run_SLAT, Cal_Spatial_Net, probabilistic_match
from scSLAT.model.prematch import icp, alpha_shape
from scSLAT.metrics import global_score

In [ ]:
sc.set_figure_params(dpi_save=200, dpi=150)

In [ ]:
# parameter cells
adata1_file = ''
adata2_file = ''
metric_file = ''
emb0_file = ''
emb1_file = ''
graphs_file = ''
matching_file = ''

In [ ]:
adata1 = sc.read_h5ad(adata1_file)
adata2 = sc.read_h5ad(adata2_file)

# Parameter for dataset

In [ ]:
if 'visium' and 'DLPFC' in adata1_file:
    biology_meta = 'cell_type'
    topology_meta = 'layer_guess'
    alpha = 10
    LGCN_layer = 2
    spot_size = 5
elif 'merfish' and 'hypothalamic' in adata1_file:
    biology_meta = 'Cell_class'
    topology_meta = 'region'
    alpha = 25
    LGCN_layer = 2
    spot_size = 15
elif 'stereo' and 'embryo' in adata1_file:
    biology_meta = 'annotation'
    topology_meta = 'region'
    alpha = 3
    LGCN_layer = 1
    spot_size = 5
elif 'brain' in adata1_file:
    biology_meta = 'layer_guess'
    topology_meta = 'layer_guess'
    alpha = 10
    LGCN_layer = 2
    spot_size = 5

# Prematch

In [ ]:
start = time.time()
if 'rotation' in adata2.uns.keys():
    boundary_1, edges_1, _ = alpha_shape(adata1.obsm['spatial'], alpha=alpha, only_outer=True)
    boundary_2, edges_2, _ = alpha_shape(adata2.obsm['spatial'], alpha=alpha, only_outer=True)
    T, error = icp(adata2.obsm['spatial'][boundary_2,:].T, adata1.obsm['spatial'][boundary_1,:].T)
    rotation = np.arcsin(T[0,1]) * 360 / 2 / np.pi

    print("T",  T)
    print("icp loss", error)
    print("rotation°", rotation)

    print(f"ground truth: {adata2.uns['rotation']}, prematch result:{rotation}, error is {adata2.uns['rotation'] - rotation}")

    trans = np.squeeze(cv2.transform(np.array([adata2.obsm['spatial']], copy=True).astype(np.float32), T))[:,:2]
    adata2.obsm['spatial'] = trans

# run SLAT with DPCA

In [ ]:
Cal_Spatial_Net(adata1, k_cutoff=20, model='KNN')
Cal_Spatial_Net(adata2, k_cutoff=20, model='KNN')
edges, features = load_anndatas([adata1, adata2], feature='dpca', singular=True, dim=30)
embd0, embd1, time1 = run_SLAT(features, edges, 6, LGCN_layer=LGCN_layer)
run_time = str(time.time() - start)
print('Runtime: ' + run_time)

In [ ]:
adata1.obsm['X_slat'] = embd0.cpu().detach().numpy()
adata2.obsm['X_slat'] = embd1.cpu().detach().numpy()

# Metric

In [ ]:
embd0 = adata1.obsm['X_slat']
embd1 = adata2.obsm['X_slat']
matching = probabilistic_match(embds=[embd0, embd1], adatas = [adata1,adata2])

In [ ]:
overall_score = global_score([adata1,adata2], matching, biology_meta, topology_meta)
celltype_score = global_score([adata1,adata2], matching, biology_meta=biology_meta)
region_score = global_score([adata1,adata2], matching, topology_meta=topology_meta)

# Save

In [ ]:
metric_dic = {}
metric_dic['global_score'] = overall_score
metric_dic['celltype_score'] = celltype_score
metric_dic['region_score'] = region_score
metric_dic['run_time'] = run_time
metric_dic['euclidean_dis'] = -1
metric_dic['angle_delta'] = -1

metric_dic['celltype_macro_f1'] = -1
metric_dic['celltype_micro_f1'] = -1
metric_dic['region_macro_f1'] = -1
metric_dic['region_micro_f1'] = -1
metric_dic['total_macro_f1'] = -1
metric_dic['total_micro_f1'] = -1

metric_dic['match_ratio'] = -1

with open(metric_file, "w") as f:
    yaml.dump(metric_dic, f)

np.savetxt(emb0_file, adata1.obsm['X_slat'], delimiter=',')
np.savetxt(emb1_file, adata2.obsm['X_slat'], delimiter=',')
# np.savetxt(matching_file, matching, fmt='%i')
with open(matching_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(matching)

In [ ]:
# save graphs for edge score
edges = [edge.cpu().detach() for edge in edges]
with open(graphs_file, 'wb') as f:
    cpickle.dump(edges, f)